# Hourly accumulations

This notebook takes the scans within the hour, converts them to rain rate then to rainfall amount, then adds it up until the end of the hour. The hour totals are saved in a hdf file.

Import libraries.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%pylab inline

import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os
import numpy as np

import pickle

import pytz
import datetime as dt

from copy import deepcopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

import boto3
from botocore.handlers import disable_signing

Populating the interactive namespace from numpy and matplotlib

## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



Read the shapefiles.

In [3]:
reader = shpreader.Reader(r'/home/ac.jcorner/Rainfall/tl_2016_17_cousub/tl_2016_17_cousub.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

/home/ac.jcorner/anaconda3/envs/rainfall/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Read the gauge locations file and gauge observations file into pandas dataframes, and get the location coordinates.

In [42]:
# load CCN gauge locations
CCN_gauge_locations_fname = 'C:/Users/iac6311/Documents/Work/Data/Cook_County/CookCounty_gage_locations.csv'
# load CCN gauge observations
CCN_gauge_observations_fname = 'C:/Users/iac6311/Documents/Work/Data/Cook_County/WaterYear2013.csv'

df_gauge_loc = pd.read_csv(CCN_gauge_locations_fname,header=0)
df_gauge = pd.read_csv(CCN_gauge_observations_fname,header=0)

x = df_gauge_loc['Longitude - West'].values
y = df_gauge_loc['Latitude'].values

In [43]:
def rounder(t):
    """
    Rounds the time to the nearest hour.
    """
    if t.minute >= 30:
        return t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [44]:
path_rrfiles = r'C:\Users\iac6311\Documents\Work\SAVEUR\Processed_RainRetrievals\2013\04\17'

In [45]:
rrfiles = os.listdir(path_rrfiles)

In [46]:
rrfile = rrfiles[0]

radar = pyart.io.read_cfradial(os.path.join(path_rrfiles,rrfile))

In [47]:
gatefilter = pyart.filters.GateFilter(radar)
# Develop your gatefilter first
# exclude masked gates from the gridding
#gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_transition()
gatefilter.exclude_masked('reflectivity')

In [48]:
radar.fields.keys()

dict_keys(['velocity', 'differential_reflectivity', 'reflectivity', 'differential_phase', 'cross_correlation_ratio', 'spectrum_width', 'corrected_reflectivity', 'normalized_coherent_power', 'unfolded_differential_phase', 'corrected_differential_phase', 'specific_differential_phase', 'attenuation', 'rainrate_from_kdp', 'rainrate_from_attenuation', 'rainrate'])

In [49]:
# perform Cartesian mapping, limit to the reflectivity field.
grid = pyart.map.grid_from_radars(
    [radar], gatefilters=[gatefilter],
    grid_shape=(1, 600, 600),
    grid_limits=((2000, 2000), (-123000.0, 123000.0), (-123000.0, 123000.0)),
    fields=['rainrate','rainrate_from_attenuation','rainrate_from_kdp'])

In [50]:
x_rad, y_rad = grid.get_point_longitude_latitude()

In [51]:
# 3.2 Get radar data
# Get slice
radar_slice0 = radar.get_slice(0)
rr_0 = radar.fields['rainrate']['data'][radar_slice0, :]

In [52]:
sitecoords = (radar.longitude['data'][0],radar.latitude['data'][0],radar.altitude['data'][0])
az = radar.azimuth['data'][radar_slice0]
r = radar.range['data']
proj = wrl.georef.epsg_to_osr(4326)

In [53]:
# save the radar parameters
with open('radarparams600x600.pkl','wb') as f:
    pickle.dump([sitecoords, az, r],f)

In [54]:
radar_depth = wrl.trafo.r_to_depth(rr_0,interval=256)

In [55]:
# create an empty dictionary
fname_dict = {}
for i in arange(1,25,1):
    fname_dict[i] = []
# fill in dictionary
for i in arange(len(rrfiles)):
    fname = rrfiles[i]
    dtime_utc = dt.datetime.strptime(fname,'radar_KLOT_%Y%m%d_%H%M%S.nc')
    fname_dict[dtime_utc.hour+1].append(dt.datetime.strftime(dtime_utc,'%Y/%m/%d/')+rrfiles[i])

In [56]:
fname_dict

{1: ['2013/04/17/radar_KLOT_20130417_000444.nc',
  '2013/04/17/radar_KLOT_20130417_001029.nc',
  '2013/04/17/radar_KLOT_20130417_001615.nc',
  '2013/04/17/radar_KLOT_20130417_002201.nc',
  '2013/04/17/radar_KLOT_20130417_002746.nc',
  '2013/04/17/radar_KLOT_20130417_003331.nc',
  '2013/04/17/radar_KLOT_20130417_003916.nc',
  '2013/04/17/radar_KLOT_20130417_004500.nc',
  '2013/04/17/radar_KLOT_20130417_005048.nc',
  '2013/04/17/radar_KLOT_20130417_005633.nc'],
 2: ['2013/04/17/radar_KLOT_20130417_010220.nc',
  '2013/04/17/radar_KLOT_20130417_010806.nc',
  '2013/04/17/radar_KLOT_20130417_011350.nc',
  '2013/04/17/radar_KLOT_20130417_012138.nc',
  '2013/04/17/radar_KLOT_20130417_012724.nc',
  '2013/04/17/radar_KLOT_20130417_013311.nc',
  '2013/04/17/radar_KLOT_20130417_013856.nc',
  '2013/04/17/radar_KLOT_20130417_014442.nc',
  '2013/04/17/radar_KLOT_20130417_015026.nc',
  '2013/04/17/radar_KLOT_20130417_015613.nc'],
 3: ['2013/04/17/radar_KLOT_20130417_020158.nc',
  '2013/04/17/radar_KLO

In [59]:
for hour in list(fname_dict)[23:]:
    print(hour)
    fnames_within_hour = fname_dict[hour]
    hour_accum_rrZ = np.zeros((600,600))
    hour_accum_rrA = np.zeros((600,600))
    hour_accum_rrKDP = np.zeros((600,600))
    for fname in fnames_within_hour:
        print('.',end='')

        # get local time of radar
        fname = fname.rsplit('/',1)[-1]
        dtime_utc = dt.datetime.strptime(fname,'radar_KLOT_%Y%m%d_%H%M%S.nc')
        dtime_utc = pytz.utc.localize(dtime_utc)
        
        # read radar data
        radar = pyart.io.read_cfradial(os.path.join(path_rrfiles,fname))

        # grid 
        gatefilter = pyart.filters.GateFilter(radar)
        gatefilter.exclude_transition()
        gatefilter.exclude_masked('reflectivity')

        # perform Cartesian mapping, limit to the reflectivity field.
        grid = pyart.map.grid_from_radars(
            [radar], gatefilters=[gatefilter],
            grid_shape=(1, 600, 600),
            grid_limits=((2000, 2000), (-123000.0, 123000.0), (-123000.0, 123000.0)),
            fields=['rainrate','rainrate_from_kdp'])

        # 3.2 Get radar data
        # Get slice
        rrZ_0 = grid.fields['rainrate']['data']
        #rrA_0 = grid.fields['rainrate_from_attenuation']['data']
        rrKDP_0 = grid.fields['rainrate_from_kdp']['data']
        
        # convert rain rate to rain amount
        rrZ_amount = wrl.trafo.r_to_depth(rrZ_0,interval=345)
        #rrA_amount = wrl.trafo.r_to_depth(rrA_0,interval=345)
        rrKDP_amount = wrl.trafo.r_to_depth(rrKDP_0,interval=345)
        
        hour_accum_rrZ += rrZ_amount[0]
        #hour_accum_rrA += rrA_amount[0]
        hour_accum_rrKDP += rrKDP_amount[0]

    savefname = dt.datetime.strftime(dtime_utc.replace(microsecond=0,second=0,minute=0)+dt.timedelta(hours=1), '%Y%m%d_%H%M%S')
    wrl.io.to_hdf5('gridded_600x600_KLOT'+savefname+'_rrZ.hdf5', hour_accum_rrZ)
    #wrl.io.to_hdf5('gridded_600x600_KLOT'+savefname+'_rrA.hdf5', hour_accum_rrA)
    wrl.io.to_hdf5('gridded_600x600_KLOT'+savefname+'_rrKDP.hdf5', hour_accum_rrKDP)

    print('')

24
..............


In [58]:
x_rad, y_rad = grid.get_point_longitude_latitude()

# Saving the objects:
with open('radar_grid_600x600.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([x_rad, y_rad], f)